# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
#Locate and load CSV data from WeatherPy exercise
City_Data_File = os.path.join('..','WeatherPy','output_data','cities.csv')
City_Data = pd.read_csv(City_Data_File)
City_Data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,San Vicente,-26.6167,-54.1333,57.81,76,0,2.55,AR,1619935121
1,1,Rikitea,-23.1203,-134.9692,77.00,76,89,14.70,PF,1619935121
2,2,Dikson,73.5069,80.5464,27.82,86,98,4.90,RU,1619935122
3,3,Punta Arenas,-53.1500,-70.9167,39.20,81,75,14.97,CL,1619935122
4,4,Talovaya,51.1144,40.7300,59.32,64,95,6.24,RU,1619935123


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
#Configure gmaps
gmaps.configure(api_key = g_key)

In [65]:
# Store 'Lat' and 'Lng' into  locations and Humidity into Weight
Locations = City_Data[["Lat", "Lng"]].astype(float)
Humidity = City_Data["Humidity"].astype(float)
Max_Humidity = Humidity.max()

#Add Heatmap layer to map.
Updated_fig = gmaps.figure()

Heat_Layer = gmaps.heatmap_layer(Locations, weights = Humidity, 
                                 dissipating = False, max_intensity = Max_Humidity,
                                 point_radius = 5)

Updated_fig.add_layer(Heat_Layer)

Updated_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
#Narrow down the cities to fit weather conditions of: Max Temp ranges, Wind Speeds, Cloudiness
#Drop any rows with nulls
Preferred_Cities_List = City_Data.loc[(City_Data["Max Temp"] > 70) & (City_Data["Max Temp"] < 85) & (City_Data["Wind Speed"] < 9) & (City_Data['Cloudiness'] < 5)]
Preferred_Cities_List = Preferred_Cities_List.dropna(how = 'any')

Preferred_Cities_List

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
109,109,Murgab,37.4966,61.9714,80.6,34,0,6.91,TM,1619935167
156,156,Saint-Philippe,-21.3585,55.7679,84.2,54,0,3.44,RE,1619935185
221,221,Saint-Pierre,-21.3393,55.4781,84.2,54,0,3.44,RE,1619935214
232,232,Kilis,36.7161,37.1150,73.4,31,0,7.63,TR,1619935219
277,277,Tahoua,14.8888,5.2692,78.8,14,0,2.30,NE,1619935236
295,295,Saint Anthony,45.0205,-93.2180,73.4,27,1,6.91,US,1619935244
450,450,Şile,41.1754,29.6133,71.6,60,0,4.61,TR,1619935307
453,453,Saint-Joseph,-21.3667,55.6167,84.2,54,0,3.44,RE,1619935308


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
#store in hotel_df and add a column for Hotel Name
hotel_df = Preferred_Cities_List
hotel_df["Hotel Name"] = ""

hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
109,109,Murgab,37.4966,61.9714,80.6,34,0,6.91,TM,1619935167,
156,156,Saint-Philippe,-21.3585,55.7679,84.2,54,0,3.44,RE,1619935185,
221,221,Saint-Pierre,-21.3393,55.4781,84.2,54,0,3.44,RE,1619935214,
232,232,Kilis,36.7161,37.1150,73.4,31,0,7.63,TR,1619935219,
277,277,Tahoua,14.8888,5.2692,78.8,14,0,2.30,NE,1619935236,
295,295,Saint Anthony,45.0205,-93.2180,73.4,27,1,6.91,US,1619935244,
450,450,Şile,41.1754,29.6133,71.6,60,0,4.61,TR,1619935307,
453,453,Saint-Joseph,-21.3667,55.6167,84.2,54,0,3.44,RE,1619935308,


In [55]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame

# find the closest hotel of each City

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get hotel location from df
    Lat = row['Lat']
    Lng = row['Lng']

    # add location to params dict
    params['location'] = f"{Lat},{Lng}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "***No Hotel within params identified***"

hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
109,109,Murgab,37.4966,61.9714,80.6,34,0,6.91,TM,1619935167,***No Hotel within params identified***
156,156,Saint-Philippe,-21.3585,55.7679,84.2,54,0,3.44,RE,1619935185,Les Embruns Du Baril
221,221,Saint-Pierre,-21.3393,55.4781,84.2,54,0,3.44,RE,1619935214,"Villa Delisle, Hotel & Spa"
232,232,Kilis,36.7161,37.1150,73.4,31,0,7.63,TR,1619935219,Olea Hotel
277,277,Tahoua,14.8888,5.2692,78.8,14,0,2.30,NE,1619935236,Hôtel Mariane
295,295,Saint Anthony,45.0205,-93.2180,73.4,27,1,6.91,US,1619935244,Hewing Hotel
450,450,Şile,41.1754,29.6133,71.6,60,0,4.61,TR,1619935307,Phellos Suites
453,453,Saint-Joseph,-21.3667,55.6167,84.2,54,0,3.44,RE,1619935308,"""Plantation Bed and Breakfast"


In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [66]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
Updated_fig.add_layer(markers)

# Display figure
Updated_fig

Figure(layout=FigureLayout(height='420px'))

***Note: 3 markers clustered near Madagascar***